In [ ]:
import numpy as np
from qiskit import QuantumCircuit, transpile
from qiskit.providers.aer import AerSimulator
from qiskit.providers.aer.noise import NoiseModel, depolarizing_error
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve, auc, accuracy_score
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [ ]:
circ = QuantumCircuit(2,2)
circ.x(0); circ.x(1)
circ.barrier()
circ.measure([0,1],[0,1])

noise_levels = np.linspace(0.01, 0.10, 5)
shots = 512
data, labels, noise_list = [], [], []

for p in noise_levels:
    nm = NoiseModel()
    err = depolarizing_error(p, 2)
    nm.add_all_qubit_quantum_error(err, ['x','cx'])
    sim = AerSimulator(noise_model=nm)
    for _ in range(20):
        res = sim.run(transpile(circ, sim), shots=shots).result()
        counts = res.get_counts()
        probs = [counts.get(s,0)/shots for s in ['00','01','10','11']]
        fidelity = counts.get('11',0)/shots
        data.append(probs)
        labels.append(int(fidelity>=0.8))
        noise_list.append(p)

X = np.array(data)
y = np.array(labels)